In [1]:
#You have to put the spawn protection in 0 so you can play normally(this inside Server.properties)
#import the libraries needed for the server to run

import requests

import time

import os

import re

import json

import glob

import threading

import subprocess


#Update apt cache

!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

#Asign the files to the directory

os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #List the files in the directory to verify them



#Import the configuration file

config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Use the default configuration if there the configuration file does not exist

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)



# Instal OpenJDK

version = colabconfig["server_version"]

def installjava17(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # CHECK if the command was executed successfully
    if proceso.returncode == 0:
        print("OpenJDK 17 instaled correctly")
    else:
        print("Failed to install OpenJDK 17")

def installjava11(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # CHECK if the command was executed successfully
    if proceso.returncode == 0:
        print("OpenJDK 11 instaled correctly")
    else:
        print("Failed to install OpenJDK 11")

def installjava8(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # CHECK if the command was executed successfully
    if proceso.returncode == 0:
        print("OpenJDK 8 instaled correctly")
    else:
        print("Failed to install OpenJDK 8")
        
def purgejava(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # CHECK if the command was executed successfully
    if proceso.returncode == 0:
        print("OpenJDK purged correctly")
    else:
        print("Failed to purge OpenJDK")

# Check the version of the server and install the corresponding version of OpenJDK

if version < "1.14":
    # Purge previous versions of OpenJDK
    purgejava('sudo apt-get purge openjdk* -y')
    # INSTALL openJDK 8
    installjava8('sudo apt-get install openjdk-8-jre-headless -y')

elif version >= "1.14" and version < "1.17":
    # Purge previous versions of OpenJDK
    purgejava('sudo apt-get purge openjdk* -y')
    # INSTALL openJDK 11
    installjava11('sudo apt-get install openjdk-11-jre-headless -y')

else:
    # Purge previous versions of OpenJDK
    purgejava('sudo apt-get purge openjdk* -y')
    # INSTALL openJDK 17
    installjava17('sudo apt-get install openjdk-17-jre-headless -y')

# Check the java version

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

#List of the server types and their respective jar files

jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar", 'catserver': "catserver.jar", 'purpur': "purpur.jar"}

jar_name = jar_list[colabconfig["server_type"]]




# Java arguments.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No flags for other server types

#JVM Arguments for the server

memory_allocation = "-Xms10G -Xmx10G"



# tunnel_service for the server

tunnel_service = "ngrok"

print("Using: ", tunnel_service)


if tunnel_service == "ngrok":

    !pip -q install pyngrok  #Aqui se instala el ngrok en el servidor

    from pyngrok import conf, ngrok



    # Obten tu token de ngrok

    print("Consigue tu authtoken de https://dashboard.ngrok.com/auth") #Consigue aqui tu token de ngrok, sera dinamico, si quieres que sea estatico, necesitaras la version premiun de ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "2dNDenxALM33SRjENRDopFAABry_2pYXLZFtZGeXV1Fp4Zm8m"

    ! ngrok authtoken $authtoken # logeo en ngrok





    #Choose your region



    #v - - - - - - - Available Regions - - - - - - - v

    # ap - Asia/Pacific (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - South america (São Paulo - Brasil)

    # us - United estates (Ohio)

    conf.get_default().region = 'us' # <--- Place your region here


    # Conect to ngrok

    url = ngrok.connect(25565, 'tcp')

    print('Your server IP: ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #If you got the premiun version of ngrok use the comented code below to get a subdomain

    #subdomain = "my-subdomain" # <--- Place your subdomain here inside the quotes

    #url = ngrok.connect(25565, 'tcp', subdomain=subdomain)

    #print('Your server ip is:  ' + url.replace('tcp://', ''))

#If using playit.gg

elif tunnel_service == "playit":

    subprocess.run(["curl", "-SsL", "https://playit-cloud.github.io/ppa/key.gpg"], check=True)
    subprocess.run(["sudo", "apt-key", "add", "key.gpg"], check=True)

    # ADD PLAYIT.GG REPOSITORY
    subprocess.run(["sudo", "curl", "-SsL", "-o", "/etc/apt/sources.list.d/playit-cloud.list", "https://playit-cloud.github.io/ppa/playit-cloud.list"], check=True)

    # UPDATE PACKAGE LIST
    subprocess.run(["sudo", "apt", "update"], check=True)

    # INSTALL PLAYIT.GG
    result = subprocess.run(["sudo", "apt", "install", "playit"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # cHECK if the command was executed successfully
    if result.returncode == 0:
        print("Playit.gg installed")
    else:
        print("Failed to install Playit.gg")

    # Start the server
    print('Starting server...')
    subprocess.run(["playit", "&", "java", "$memory_allocation", "$server_flags", "-jar", "$jar_name", "nogui"], check=True)

    # Get the server IP
    result = subprocess.run(["playit", "status"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Tu ip es: " + result.stdout.decode())

print('Launching server...')

def get_users():
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Name: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error making the request. Status code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Conexion Error: {e}")

# Funcion for executing get_users() every 10 minutes

def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Wait 10 minutes

# Start the thread for repeat_get_users()

user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

#Obtener el archivo de configuracion
stype = colabconfig["server_type"]
version = colabconfig["server_version"]

#Imprimir el tipo de servidor y la version
print("You are currently playing to minecraft : " + stype + " on its version: "+ version)

#Si el servidor es de tipo Forge antiguo no va a funcionar
if colabconfig["server_type"] == "forge" and colabconfig["server_version"] < "1.17.1":
    print("This forge version is not compatible with the script :(.")


#If the server is a forge server, this will run the server with the forge arguments

if colabconfig["server_type"] == "forge":

    version = colabconfig["server_version"]
    
    forgefiles = f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/"
    
    forgepreversion = os.listdir(forgefiles)
    
    if forgepreversion:
    
        forgeversion = forgepreversion[0]
        forgeversionchecked = forgeversion.replace(".jar", "")
        print("Your forge version is: ", forgeversionchecked)
    
    else:
    
        print("There was not any forge files.")
    
    pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/{forgeversionchecked}/unix_args.txt")

    if pathtoforge: 

        path = pathtoforge[0] 

        print(path)

        !java @user_jvm_args.txt "@{path}" "$@"

    else:

        print("No unix_args.txt found.")


else:
    
    !java $memory_allocation $server_flags -jar $jar_name nogui   


# Cicle to keep the server running

while True:
    time.sleep(60) #Wait 60 seconds











apt cache successfully updated
banned-ips.json      config	permissions.yml    version_history.json
banned-players.json  eula.txt	plugins		   versions
bukkit.yml	     help.yml	server.jar	   whitelist.json
cache		     libraries	server.properties  world
colabconfig.json     logs	spigot.yml	   world_nether
commands.yml	     ops.json	usercache.json	   world_the_end
OpenJDK purged correctly
OpenJDK 17 instaled correctly
Using:  ngrok
Consigue tu authtoken de https://dashboard.ngrok.com/auth
Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml
Your server IP: 8.tcp.ngrok.io:17334
Launching server...
You are currently playing to minecraft : paper on its version: 1.20.4
Lista de usuarios:
Name: Leanne Graham, Email: Sincere@april.biz
Name: Ervin Howell, Email: Shanna@melissa.tv
Name: Clementine Bauch, Email: Nathan@yesenia.net
Name: Patricia Lebsack, Email: Julianne.OConner@kory.org
Name: Chelsey Dietrich, Email: Lucio_Hettinger@annie.ca
Name: Mrs. Dennis Schulist, Ema